# Jupyter Canvas Widget - Rail Editor

From - https://github.com/Who8MyLunch/Jupyter_Canvas_Widget/blob/master/notebooks/example%20mouse%20events.ipynb
Follow his instructions to do a local dev install and enable the widget.

## You need to run all cells before trying to edit the rails!

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import image_attendant as imat
import ipywidgets
import IPython
import jpy_canvas
import numpy as np
from numpy import array
import time
from collections import deque
from matplotlib import pyplot as plt
import io
from PIL import Image

In [3]:
from ipywidgets import IntSlider, link, VBox, RadioButtons, HBox, interact

In [4]:
import flatland.core.env
from flatland.envs.rail_env import RailEnv, random_rail_generator
from flatland.core.transitions import RailEnvTransitions
from flatland.core.env_observation_builder import TreeObsForRailEnv
import flatland.utils.rendertools as rt
from flatland.utils.editor import JupEditor

cpu


In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [6]:
oEnv = RailEnv(width=20,
              height=20,
              rail_generator=random_rail_generator(cell_type_relative_proportion=[1,1] + [0.5] * 6),
              number_of_agents=0,
              obs_builder_object=TreeObsForRailEnv(max_depth=2))
obs = oEnv.reset()

In [7]:
sfEnv = "../flatland/env-data/tests/test1.npy"
if True:
    oEnv.rail.load_transition_map(sfEnv)
    oEnv.width = oEnv.rail.width
    oEnv.height = oEnv.rail.height

In [8]:
oRT = rt.RenderTool(oEnv)

In [9]:
oEnv.width

10

In [10]:
import sys
sys.path

['/home/jeremy/projects/heating',
 '/home/jeremy/projects/aicrowd/rl-trains/MAgent/python',
 '/home/jeremy/ve367/lib/python36.zip',
 '/home/jeremy/ve367/lib/python3.6',
 '/home/jeremy/ve367/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '',
 '/home/jeremy/ve367/lib/python3.6/site-packages',
 '/home/jeremy/projects/aicrowd/rl-trains/torch-rl/torch_rl',
 '/home/jeremy/projects/aicrowd/rl-trains/gym-minigrid',
 '/home/jeremy/projects/aicrowd/rl-trains/flatland',
 '/home/jeremy/projects/aicrowd/rl-trains/Jupyter_Canvas_Widget',
 '/home/jeremy/ve367/lib/python3.6/site-packages/IPython/extensions',
 '/home/jeremy/.ipython']

### Clear the rails

In [11]:
oEnv.rail.grid[:,:] = 0

### Render the env in the usual way, and take an image snapshot as numpy array
If you have already edited the env in the cell below, these changes should be reflected here.

In [12]:
oFig = plt.figure(figsize=(10,10))
oRT.renderEnv(spacing=False, arrows=False, sRailColor="gray", show=False)
img = oRT.getImage()
print(type(img))
plt.clf()   # if you don't want the image to appear here
pass

wid_img = jpy_canvas.Canvas(img)

<class 'numpy.ndarray'>


<Figure size 720x720 with 0 Axes>

### Update the function - in case external code updated

In [13]:
wid_img.unregister_all()
oEditor = JupEditor(oEnv, wid_img)
wid_img.register_move(oEditor.event_handler)
wid_img.register_click(oEditor.on_click)


Correct editor


### Some more widgets

In [14]:
wid_drawMode = ipywidgets.RadioButtons(options=["Draw", "Erase", "Origin", "Destination"])
wid_drawMode.observe(oEditor.setDrawMode, names="value")
wid_debug = ipywidgets.Checkbox(description = "Debug")
wid_debug.observe(oEditor.setDebug, names="value")
wid_debug_move = ipywidgets.Checkbox(description = "Debug mouse move")
wid_debug_move.observe(oEditor.setDebugMove, names="value")

wid_output = ipywidgets.Output()
oEditor.setOutput(wid_output)
wid_filename = ipywidgets.Text(description = "Filename")
wid_filename.value = sfEnv
oEditor.setFilename(sfEnv)
wid_filename.observe(oEditor.setFilename_event, names="value")

wid_size = ipywidgets.IntSlider(min=5, max=30, step=5, description="Regen Size")
wid_size.observe(oEditor.setRegenSize_event, names="value")

prog_steps = ipywidgets.IntProgress(value=0, min=0, max=20, step=1, description="Step")
#prog_steps.observe(oEditor.)


ldButtons = [
    dict(name = "Refresh", method = oEditor.redraw_event),
    dict(name = "Clear", method = oEditor.clear),
    dict(name = "Regenerate", method = oEditor.regenerate_event),
    dict(name = "Load", method = oEditor.load),
    dict(name = "Save", method = oEditor.save),
    dict(name = "Step", method = oEditor.step_event),
    dict(name = "Run Steps", method = oEditor.start_run_event),
]

lwid_buttons = []
for dButton in ldButtons:
    wid_button = ipywidgets.Button(description = dButton["name"])
    wid_button.on_click(dButton["method"])
    lwid_buttons.append(wid_button)
    

#wid_debug = interact(oEditor.setDebug, debug=False)
vbox_controls = VBox([wid_filename, wid_drawMode, *lwid_buttons, wid_size, wid_debug, wid_debug_move])


### Edit the map below here by dragging the mouse to create transitions
You can create a dead-end by dragging foward and backward, ie Cell A -> Adjacent Cell B -> back to Cell A

In [15]:
# wid_box
wid_main = HBox([wid_img, vbox_controls])
wid_output.clear_output()
wid_main


In [16]:
wid_output.clear_output()
wid_output

Output()

### Experimental
enable the thread.start below to try background stepping of the agents.  It tends to make a mess.

In [17]:
import threading
def bgUpdate(editor):
    for i in range(100):
        editor.step_event()
        time.sleep(0.2)


In [18]:
thread = threading.Thread(target=bgUpdate, args = (oEditor,))

In [19]:
if False:
    thread.start()

### Save the image (change the filename...)

In [20]:
if False: 
    oEnv.rail.save_transition_map("../flatland/env-data/tests/test-editor.npy")

## Junk below here

In [21]:
def evListen(wid, ev):
    x = ev["canvasX"]
    y=ev["canvasY"]
    yxBase = array([6, 21])
    nPixCell = 35
    rcCell = ((array([y, x]) - yxBase) / nPixCell).astype(int)
    print(ev)
    print(x, y, (x-21) / 35, (y-6) / 35, rcCell)

In [22]:
# wid_img.register_click(evListen)
#wid_img.register(evListen)

In [23]:
#wid_img.unregister_all()

In [24]:
ipywidgets.IntText(0)

IntText(value=0)

In [25]:
from examples.play_model import Player

In [26]:
oP = Player(oEnv)

In [27]:
oP.step()

In [28]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [29]:
slider = ipywidgets.IntSlider()

async def f():
    for i in range(10):
        print('did work %s'%i)
        x = await wait_for_change(slider, 'value')
        print('async function continued with value %s'%x)
asyncio.ensure_future(f())

slider

IntSlider(value=0)

In [30]:

import threading
from IPython.display import display
import time
progress = ipywidgets.FloatProgress(value=0.0, min=0.0, max=1.0)

def work(progress):
    total = 100
    for i in range(total):
        time.sleep(0.2)
        progress.value = float(i+1)/total

thread = threading.Thread(target=work, args=(progress,))
display(progress)
thread.start()


FloatProgress(value=0.0, max=1.0)

In [31]:
binTrans = format(32800, "#016b")
[binTrans[i+2:i+6] for i in range(0, len(binTrans)-2, 4)]

['1000', '0000', '0010', '0000']

In [32]:
oEnv.rail

In [33]:
binDeadend = 0b0010000110000100
format(~binDeadend & 0xffff, "#018b")

'0b1101111001111011'

In [34]:
oEnv.rail.cell_neighbours_valid((2,5))

True

In [35]:
oEnv.rail.grid[2,6] & binDeadend

0

In [36]:
oTrans = oEnv.rail.grid[2,5]
oTrans
oEnv.rail.transitions.repr(oTrans, 1)

''

In [37]:
oEnv.rail.transitions.is_valid(oTrans)

True

In [38]:
oEnv.rail.transitions.gDir2dRC

array([[-1,  0],
       [ 0,  1],
       [ 1,  0],
       [ 0, -1]])

In [39]:
binTrans = oEnv.rail.get_transitions((2,2))
sBinTrans = format(binTrans, "#018b")[2:]
print(sBinTrans)
lsBinTrans = [ sBinTrans[i:i+4] for i in range(0,16, 4)]
print(lsBinTrans)

0000000000000000
['0000', '0000', '0000', '0000']


In [40]:
lnBinTrans = array([ binTrans >> 8, binTrans & 0xff ], dtype=np.uint8)
g2binTrans = np.unpackbits(lnBinTrans).reshape(4,4)
g2binTrans

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=uint8)

In [41]:
gDirIn = g2binTrans.any(axis=1)
gDirOut = g2binTrans.any(axis=0)
print(gDirIn)
print(gDirOut)

[False False False False]
[False False False False]


In [42]:
giDirOut = np.argwhere(gDirOut)[:,0]
giDirOut

array([], dtype=int64)

In [43]:
any((0,0,0))

False

In [44]:
oEnv.rail.transitions.gDir2dRC

array([[-1,  0],
       [ 0,  1],
       [ 1,  0],
       [ 0, -1]])

In [45]:
oEnv.rail.transitions.gDir2dRC[giDirOut,:]

array([], shape=(0, 2), dtype=int64)

In [46]:
t4TransOut = oEnv.rail.get_transitions((2,2,3))
print (t4TransOut)                                        
for iDir in np.argwhere(t4TransOut)[:,0]:
    print(iDir, oEnv.rail.transitions.gDir2dRC[iDir])
    

(0, 0, 0, 0)


In [47]:
lsRepr = []
sDirs = "NESW"
lsDirs = list(sDirs)
cell_transition = oTrans
sbinTrans = format(cell_transition, "#018b")[2:]
print(sbinTrans)
for iDirIn in range(0,4):
    sDirTrans = sbinTrans[iDirIn*4:iDirIn*4+4]
    if sDirTrans=="0000":
        continue
    sDirsOut = [
        lsDirs[iDirOut]
        for iDirOut in range(0,4) 
        if sDirTrans[iDirOut]=="1"
        ]
    lsRepr.append(lsDirs[iDirIn] + ":" + "".join(sDirsOut))

", ".join(lsRepr)

0000000000000000


''

In [48]:
s="1010"
ints = int(s,2)
gs = array(ints, dtype=np.uint8)
print(gs, np.unpackbits(gs)[4:8])


10 [1 0 1 0]


In [49]:
oEnv.rail.transitions.repr(1025)

'N:0000 E:0100 S:0000 W:0001'